# Qiskit Runtime REST API query & job submission

## (optional) get temporary Access token from Auth API via API Token

This is especially useful if you would like control over tokens, such as token invalidation. Alternatively, you can work directly with your IBM Quantum Platform API token.

In [1]:
import requests

with open('token') as file:
    token=file.read()

url = 'https://auth.quantum-computing.ibm.com/api/users/loginWithToken'
input={'apiToken': token}
auth_response = requests.post(url, json=input)
auth_id=auth_response.json()['id']

## GET available backends

In [2]:
url_backends = 'https://api.quantum-computing.ibm.com/runtime/backends'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}

backends_response = requests.get(url_backends, headers=headers)

print(backends_response.json()['devices'][:5],"...")

['ibm_kyoto', 'ibmq_qasm_simulator', 'ibm_sherbrooke', 'ibm_pinguino3', 'ibm_pinguino23'] ...


## Run a job

### set up qasm circuit

In [53]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[2];
creg c[2];
x q[0];
c[0] = measure q[0]; 
c[1] = measure q[1];
'''   

## transpile circuit via transpilation service API

Query the cloud Transpilation service at https://cloud-transpiler.quantum-computing.ibm.com/docs . See more details and documentation at https://docs.quantum.ibm.com/transpile/qiskit-transpiler-service . Note that [transpilation is now necessary](https://docs.quantum.ibm.com/announcements/product-updates/2024-02-14-qiskit-runtime-primitives-update) prior to submitting a circuit to IBM Quantum backends. 

In [54]:
backend='ibm_hanoi'

In [55]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
        }
body= {
    "qasm_circuits": qasm_string,
}
params = {
        "backend": backend,
        "optimization_level": 1,
        "use_ai": False,
        }

resp = requests.post(
            "https://cloud-transpiler.quantum.ibm.com/transpile",
            headers=headers,
            json=body,
            params=params,
        )


In [56]:
task_id=resp.json()['task_id']
task_id

'7a38212c-efca-46a3-9a3f-899652274e7b'

### Request the transpilation service results using the task_id

In [57]:
res = requests.get(url=f"https://cloud-transpiler.quantum.ibm.com/transpile/{task_id}", headers=headers)
print(res.json())

{'state': 'SUCCESS', 'result': [{'qasm': 'OPENQASM 3.0; include "stdgates.inc"; bit[2] c; x $0; c[0] = measure $0; c[1] = measure $1;', 'success': True, 'layout': {'initial': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], 'final': [0, 1]}}]}


In [58]:
if res.json().get("state") == "SUCCESS":
    resulting_qasm=res.json().get("result")[0].get("qasm")

print(resulting_qasm)

OPENQASM 3.0; include "stdgates.inc"; bit[2] c; x $0; c[0] = measure $0; c[1] = measure $1;


### run circuit via API

Note the jobs below use Qiskit Runtime V2 primitives. C.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
Both SamplerV2 and EstimatorV2 take one or more primitive unified blocs (PUBs) as the input. Each PUB is a tuple that contains one circuit and the data broadcasted to that circuit, which can be multiple observables and parameters. Each PUB returns a result.

In [31]:
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend, 
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "start_session": "False", #set to False if you just need to run a single job.  
    "params": {
        "pubs": [[resulting_qasm],[resulting_qasm,None,500]], #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "version": 2 #this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"crw9cjj7wv80008fgja0","backend":"ibm_hanoi"}


### check job status

In [32]:
response_status_singlejob= requests.get(url+'/'+job_id, headers=headers)
response_status_singlejob.json().get('state')

{'status': 'Queued', 'reason': '', 'reasonCode': None}

### Get job results

In [12]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

res_dict=response_result.json()

counts=res_dict['results'][0]['data']['c']['samples']
print(counts[:20])

['0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1']


#### Convert counts from hexadecimal format

In [13]:
countslist=[int(x,base=16) for x in res_dict['results'][1]['data']['c']['samples']]
print(countslist[:20])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [14]:
from collections import Counter
count_counts=Counter(countslist)
count_counts

Counter({1: 496, 0: 4})

## Create Session 

c.f. documentation at https://docs.quantum.ibm.com/api/runtime

In [76]:
import json
 
sessionsUrl = "https://api.quantum-computing.ibm.com/runtime/sessions"
 
headersList = {
  "Accept": "application/json",
  # "Authorization": "Bearer "+token,
  'x-access-token':auth_id,
  "Content-Type": "application/json" 
}
 
payload = json.dumps({
  "backend": backend,
  "instance": "ibm-q-internal/dev-sys-software/internal-test"
})
 
response = requests.request("POST", sessionsUrl, data=payload,  headers=headersList)
 
sessionId = response.json()['id']
 
print(response.json())

{'id': 'crw9kjy7jqmg008zp7mg'}


### Submit job(s) against resulting Session ID

#### Submit SamplerV2 job to Session

In [77]:
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }

job_input = {
    'program_id': 'sampler',
    "backend": backend, 
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "session_id": sessionId, # This specifies the previously created Session 
    "params": {
        "pubs": [[resulting_qasm],[resulting_qasm,None,100]], #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "version": 2, #this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "options":{
                "transpilation":{"optimization_level": 1},
                "twirling": {"enable_gates": True,"enable_measure": True}, #c.f. documentation at https://docs.quantum.ibm.com/run/configure-error-mitigation#custom-error-settings-v2-primitives 
                # "dynamical_decoupling": {"enable": True, "sequence_type": "XpXm"},                        
                    },

}
        # "circuits": [resulting_qasm] #only use this input for Primitives V1, i.e. "version": 1
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id_sampler = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"crw9kkex484g008fc7ng","backend":"ibm_hanoi","session_id":"crw9kjy7jqmg008zp7mg"}


#### Submit EstimatorV2 job to Session

In [78]:
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend, 
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "session_id": sessionId, # This specifies the previously created Session
    "params": {
        "pubs": [[resulting_qasm,{"II": 1, "ZZ": 2.3},None,0.01]], #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "version": 2, #this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "options": {
            "transpilation":{"optimization_level": 1},
            "resilience": {
                    "measure_mitigation": True, #c.f. documentation at https://docs.quantum.ibm.com/run/configure-error-mitigation#custom-error-settings-v2-primitives 
                #     # "measure_noise_learning": None,
                #     # "zne_mitigation": None,
                #     # "zne": None,
                #     # "pec_mitigation": None,
                #     # "pec": None,
                #     # "layer_noise_learning": None,
                    },
            "twirling": {"enable_gates": True},
            "dynamical_decoupling": {"enable": False},#, "sequence_type": "XpXm"},
                     
                },
        # "resilience_level": 0,        
    }
        # "circuits": [resulting_qasm] #only use this input for Primitives V1, i.e. "version": 1
}
response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id_estimator = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"crw9mk27wv80008fgmf0","backend":"ibm_hanoi","session_id":"crw9kjy7jqmg008zp7mg"}


### Get the Job Results

In [60]:
response_status= requests.get(url+'/'+job_id_sampler, headers=headers)
response_status.json().get('state')

{'status': 'Completed', 'reason': '', 'reasonCode': None}

In [61]:
response_sampler = requests.get(url+'/'+job_id_sampler+'/results', headers=headers)
res_dict_sampler=response_sampler.json()

counts=res_dict_sampler['results'][0]['data']['c']['samples']
print(counts[:20])

['0x1', '0x1', '0x0', '0x0', '0x0', '0x0', '0x0', '0x1', '0x1', '0x0', '0x0', '0x0', '0x0', '0x0', '0x1', '0x0', '0x0', '0x1', '0x0', '0x0']


In [62]:
response_status= requests.get(url+'/'+job_id_estimator, headers=headers)
response_status.json().get('state')


{'status': 'Completed', 'reason': '', 'reasonCode': None}

In [63]:
response_estimator = requests.get(url+'/'+job_id_estimator+'/results', headers=headers)
res_dict_estimator=response_estimator.json()

print(res_dict_estimator['results'])


[{'data': {'evs': -1.3206712251052517, 'stds': 0.019759525689619355, 'ensemble_standard_error': 0.0200795395210333}, 'metadata': {'shots': 10016, 'target_precision': 0.01, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}}]


In [64]:
exp_val=res_dict_estimator['results'][0]['data']['evs']
print('Expectation value: ', exp_val)


Expectation value:  -1.3206712251052517


### close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users. Documentation at https://docs.quantum.ibm.com/run/run-jobs-in-session

In [74]:
closureURL="https://api.quantum-computing.ibm.com/runtime/sessions/"+sessionId+"/close"

headersList = {
  "Accept": "application/json",
  # "Authorization": "Bearer "+token,
  'x-access-token':auth_id,
}
 
payload = json.dumps({
  # "backend": backend
  "instance": "ibm-q-internal/dev-sys-software/internal-test"
})
 
closure_response = requests.request(
    "DELETE",
    closureURL, 
    headers=headersList)

print("Session closure response ok?:",closure_response.ok,closure_response.text)

Session closure response ok?: True 


## (Optional) Invalidate Token 

In [93]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 


## (Optional) Test Token Invalidation

This should yield an Authoritzation error (Error 401) once access token is invalidated

In [94]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)

if logout_response.status_code == 200:
    job_id = logout_response.json().get('id')
    print("Job created:",logout_response.text)
elif logout_response.status_code == 401:
    print("invalid credentials. Access token should be successfully invalidated.")
else:
    print(logout_response.text,"\n")
    print(f"Error: {logout_response.status_code}")

invalid credentials. Access token should be successfully invalidated.
